In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [4]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_i",
    "Features": "Chemical + Mineralogical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [5]:
df = pd.read_csv("../../../../../../data/processed/partner_i-Oficial/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [6]:
df_copy = df.drop(
    [
        # Properties
        "Blaine",
        "Initial setting time",
        "Final setting time",
        "Density",
        "#200",
        "#325",
        "CS3",
        "CS7",
        
        # Removing One-Hot encoding variables
        "Cement_Type",
        
        
        # Chemical Composition
        # Reason: Loss on Ignition is the only feature
        # that belongs to chemical composition in which was 
        # measured by a different method, namely manual
        
    ],
    axis=1,
).copy()


<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [7]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [8]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -3.442 (0.049)
MAE: -2.659 (0.042)
MAPE: -0.064 (0.001)
R2: 0.534 (0.014)


******
[TEST]
******
RMSE: -3.558 (0.192)
MAE: -2.744 (0.150)
MAPE: -0.066 (0.003)
R2: 0.498 (0.061)




<IPython.core.display.Javascript object>

In [9]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model Cross Validation RMSE Test  \
                                                                     mean   
0  Chemical + Mineralogical  Linear Regression   Repeated KFold  3.557557   

             MAE Test           MAPE Test             R2 Test            
        std      mean       std      mean       std      mean       std  
0  0.191814  2.743743  0.150004  0.065643  0.003372  0.498305  0.061316

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [10]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.992 (0.320)
MAE: -2.287 (0.292)
MAPE: -0.055 (0.007)
R2: 0.650 (0.047)


******
[TEST]
******
RMSE: -3.863 (0.431)
MAE: -2.938 (0.342)
MAPE: -0.070 (0.008)
R2: 0.378 (0.131)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression              Repeated KFold   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  3.862910  0.430778  2.937553  0.342052  0.069706  0.007957  0.377587   
1  3.557557  0.191814  2.743743  0.150004  0.065643  0.003372  0.498305   

             
        std  
0  0.130954  
1  0.061316

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [12]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -3.242 (0.263)
MAE: -2.540 (0.193)
MAPE: -0.061 (0.005)
R2: 0.568 (0.056)


******
[TEST]
******
RMSE: -4.122 (0.630)
MAE: -3.160 (0.487)
MAPE: -0.075 (0.010)
R2: 0.327 (0.173)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression              Repeated KFold   
2  Chemical + Mineralogical  Linear Regression           Time Series Split   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  3.862910  0.430778  2.937553  0.342052  0.069706  0.007957  0.377587   
1  3.557557  0.191814  2.743743  0.150004  0.065643  0.003372  0.498305   
2  4.121763  0.629604  3.159903  0.486916  0.075010  0.010326  0.327181   

             
        std  
0  0.130954  
1  0.061316  
2  0.173117

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [14]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 3.532 (0.000)
MAE: 2.759 (0.000)
MAPE: 0.066 (0.000)
R2: 0.512 (0.000)


******
[TEST]
******
RMSE: 3.241 (0.000)
MAE: 2.517 (0.000)
MAPE: 0.059 (0.000)
R2: 0.576 (0.000)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression           Out of time Split   
2  Chemical + Mineralogical  Linear Regression              Repeated KFold   
3  Chemical + Mineralogical  Linear Regression           Time Series Split   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  3.862910  0.430778  2.937553  0.342052  0.069706  0.007957  0.377587   
1  3.240668  0.000000  2.516857  0.000000  0.059074  0.000000  0.575734   
2  3.557557  0.191814  2.743743  0.150004  0.065643  0.003372  0.498305   
3  4.121763  0.629604  3.159903  0.486916  0.075010  0.010326  0.327181   

             
        std  
0  0.130954  
1  0.000000  
2  0.061316  
3  0.173117

<IPython.core.display.Javascript object>

In [16]:
pipeline.named_steps["estimator"].coef_

array([-1.28683769,  0.19278447, -0.32715734,  0.84972694,  0.27032438,
        2.11148558,  0.20735389,  1.98330745,  0.30990347, -0.14862701,
        1.16768346,  0.43427335,  0.60726801, -0.59886237,  0.1316551 ,
       -0.91319849,  0.80172154,  0.67894144,  0.27477178,  0.05385379,
        0.03184803,  0.28587754,  0.49874005,  0.12631557,  0.08362379,
        0.05360326,  1.13195924,  0.01976757,  0.18531879])

<IPython.core.display.Javascript object>

In [17]:
pipeline.named_steps["estimator"].intercept_

43.398173464561

<IPython.core.display.Javascript object>

In [18]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [19]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['CaO', 'MgO', 'Na2O', 'Al2O3', 'SiO2', 'SO3', 'K2O', 'Fe2O3',
       'Loss on Ignition', 'Insoluble Residue', 'Total C3S', 'Alpha C2S',
       'Beta C2S', 'Gamma C2S', 'C4AF', 'C3A', 'Cubic C3A', 'Orthorhombic C3A',
       'Free CaO', 'Portlandite', 'Periclase', 'Arcanite', 'Aphthitalite',
       'Gypsum', 'Bassanite', 'Anhydrite', 'Calcite', 'Dolomite', 'Quartz'],
      dtype='object')

<IPython.core.display.Javascript object>

In [20]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(29,)

<IPython.core.display.Javascript object>

In [21]:
x.columns.shape

(29,)

<IPython.core.display.Javascript object>

In [22]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [23]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
SO3,2.111486
Fe2O3,1.983307
Total C3S,1.167683
Calcite,1.131959
Al2O3,0.849727
Cubic C3A,0.801722
Orthorhombic C3A,0.678941
Beta C2S,0.607268
Aphthitalite,0.498740
Alpha C2S,0.434273


<IPython.core.display.Javascript object>

In [24]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression           Out of time Split   
2  Chemical + Mineralogical  Linear Regression              Repeated KFold   
3  Chemical + Mineralogical  Linear Regression           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  3.862910  0.430778   
1                                {"Test Size": 0.2}  3.240668  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  3.557557  0.191814   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  4.121763  0.629604   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  2.937553  0.342052  0.069706  0.007957  0.377587  0.130954  
1  2.516857  0.000000  0.059074  0.000000  0.575734  0.000000  
2  2.743743  0.150004  0.065643  0.003372  0.498305  0.061316  
3  3.159903  0.486916  0.075010  0.010326  0.327181  0.173117

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [25]:
path = (
    "../../../../../../reports/results/local_models/partner_i-oficial/all_cements/full/"
)
filename = "linear_regression_results_full_6.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [26]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/partner_i-oficial/all_cements/grouped/"
filename = "linear_regression_results_grouped_6.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [27]:
df_results_to_save

Category     Company  Data Shape Timesteps                  Features  \
                                                                             
0  Local Model  partner_i  (1226, 29)       NaN  Chemical + Mineralogical   
1  Local Model  partner_i  (1226, 29)       NaN  Chemical + Mineralogical   
2  Local Model  partner_i  (1226, 29)       NaN  Chemical + Mineralogical   
3  Local Model  partner_i  (1226, 29)       NaN  Chemical + Mineralogical   

               Model            Cross Validation  \
                                                   
0  Linear Regression  Blocking Time Series Split   
1  Linear Regression           Out of time Split   
2  Linear Regression              Repeated KFold   
3  Linear Regression           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   2.991569  0.319795  ...   
1                                {"Test Size": 0.2}   3.531899  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   3.442381  0.049157  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   3.242112  0.263212  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.649767  0.046631  3.862910  0.430778  2.937553  0.342052  0.069706   
1  0.511953  0.000000  3.240668  0.000000  2.516857  0.000000  0.059074   
2  0.534156  0.014194  3.557557  0.191814  2.743743  0.150004  0.065643   
3  0.568280  0.056244  4.121763  0.629604  3.159903  0.486916  0.075010   

              R2 Test            
        std      mean       std  
0  0.007957  0.377587  0.130954  
1  0.000000  0.575734  0.000000  
2  0.003372  0.498305  0.061316  
3  0.010326  0.327181  0.173117  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>

In [28]:
c = pd.read_csv(
    "../../../../../../reports/results/local_models/partner_i-oficial/all_cements/grouped/linear_regression_results_grouped_6.csv",
    header=[0, 1],
).rename(columns=lambda x: "" if "Unnamed" in x else x, level=1)

<IPython.core.display.Javascript object>

In [29]:
c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (Category, )                 4 non-null      object 
 1   (Company, )                  4 non-null      object 
 2   (Data Shape, )               4 non-null      object 
 3   (Timesteps, )                0 non-null      float64
 4   (Features, )                 4 non-null      object 
 5   (Model, )                    4 non-null      object 
 6   (Cross Validation, )         4 non-null      object 
 7   (Cross Validation Params, )  4 non-null      object 
 8   (RMSE Train, mean)           4 non-null      float64
 9   (RMSE Train, std)            4 non-null      float64
 10  (MAE Train, mean)            4 non-null      float64
 11  (MAE Train, std)             4 non-null      float64
 12  (MAPE Train, mean)           4 non-null      float64
 13  (MAPE Train, std)       

<IPython.core.display.Javascript object>

In [30]:
df_results_to_save.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (Category, )                 4 non-null      object 
 1   (Company, )                  4 non-null      object 
 2   (Data Shape, )               4 non-null      object 
 3   (Timesteps, )                0 non-null      float64
 4   (Features, )                 4 non-null      object 
 5   (Model, )                    4 non-null      object 
 6   (Cross Validation, )         4 non-null      object 
 7   (Cross Validation Params, )  4 non-null      object 
 8   (RMSE Train, mean)           4 non-null      float64
 9   (RMSE Train, std)            4 non-null      float64
 10  (MAE Train, mean)            4 non-null      float64
 11  (MAE Train, std)             4 non-null      float64
 12  (MAPE Train, mean)           4 non-null      float64
 13  (MAPE Train, std)       

<IPython.core.display.Javascript object>

In [31]:
df_results_to_save[df_results_to_save["Cross Validation Params"].str.contains("Date")]

Empty DataFrame
Columns: [(Category, ), (Company, ), (Data Shape, ), (Timesteps, ), (Features, ), (Model, ), (Cross Validation, ), (Cross Validation Params, ), (RMSE Train, mean), (RMSE Train, std), (MAE Train, mean), (MAE Train, std), (MAPE Train, mean), (MAPE Train, std), (R2 Train, mean), (R2 Train, std), (RMSE Test, mean), (RMSE Test, std), (MAE Test, mean), (MAE Test, std), (MAPE Test, mean), (MAPE Test, std), (R2 Test, mean), (R2 Test, std)]
Index: []

[0 rows x 24 columns]

<IPython.core.display.Javascript object>

In [32]:
df.shape

(1226, 40)

<IPython.core.display.Javascript object>